# Classify  MNIST digits using a CNN with Keras

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras import layers
from time import time
from keras.datasets import mnist
from keras.utils import to_categorical
from keras import optimizers
from keras.callbacks import ModelCheckpoint, TensorBoard
import os

Using TensorFlow backend.


In [2]:
num_nodes = 100
learning_rate = 0.001

### Load Data

Upload the training and test data using the Keras library

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data(path="mnist.npz")

### Preprocess the images

In [8]:
# Rescale pixels to be in the range [0,1]
x_train = x_train.reshape((x_train.shape[0],28,28,1))/255.0
x_test = x_test.reshape(x_test.shape[0],28,28,1)/255.0

# One-hot encode labels
y_train = to_categorical(y_train,10)
y_test = to_categorical(y_test,10)

### Define the CNN model

In [5]:
model = Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=x_train.shape[1:]))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(num_nodes, activation='relu', kernel_initializer='he_uniform'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               540900    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 542,230.0
Trainable params: 542,230.0
Non-trainable params: 0.0
_________________________________________________________________


### Compile the model

In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Training

In [7]:
# Train for 1 epoch when using a CPU.
epochs = 1

os.makedirs('saved_models', exist_ok=True)
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.hdf5', 
                               verbose=1, save_best_only=True)
tensorboard = TensorBoard(log_dir='logs/{}'.format(time()))


history = model.fit(x_train, y_train, 
          validation_data=(x_test, y_test),
          epochs=epochs, batch_size=32, callbacks=[checkpointer,tensorboard], verbose=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 59s - loss: 0.1376 - acc: 0.9585 - val_loss: 0.0595 - val_acc: 0.9801


### Load the model with the best validation loss

In [41]:
model.load_weights('saved_models/weights.best.hdf5')

### Test the model

In [42]:
# get index of predicted dog breed for each image in test set
predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in x_test]

# report test accuracy
test_accuracy = 100*np.sum(np.array(predictions)==np.argmax(y_test, axis=1))/len(predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 98.1400%


### Pipeline Metrics
Use it to define the pipeline metrics that KFP will produce for every pipeline run. Kale will associate each one of these metrics to the steps that produced them. Also, you will have to choose one these metrics as the Katib search objective metric.

In [43]:
print(test_accuracy)

98.14
